# This notebook is to write a python function to convert html to png files.

Please read this notebook, `07.Genomics_use_case_Hail_plotting_standard_vm.ipynb`, for more details about the limitations in the RW and some workarounds.

In [ ]:
# Install Puppeteer using npm (Node Package Manager)
# Puppeteer is a Node library which provides a high-level API to control Chrome or Chromium over the DevTools Protocol
!npm install puppeteer

In [ ]:
!touch ~/.bashrc

In [ ]:
%%bash
# Download and run the NVM (Node Version Manager) installation script from the official GitHub repository
curl -o- https://raw.githubusercontent.com/nvm-sh/nvm/v0.39.1/install.sh | bash

# Reload the shell configuration to ensure NVM is loaded into the current session
source ~/.bashrc

# Install the latest Long Term Support (LTS) version of Node.js using NVM
nvm install --lts

In [ ]:
%%bash
# Check the installed version of Node.js
node -v

# Check the installed version of npm (Node Package Manager)
npm -v

In [ ]:
%%writefile screenshot.js
const puppeteer = require('puppeteer');
const path = require('path');

async function html_to_png(fileName) {
  try {
    const browser = await puppeteer.launch({
      headless: true,
      args: ['--no-sandbox', '--disable-setuid-sandbox']
    });

    const page = await browser.newPage();
    await page.setViewport({ width: 100, height: 600 }); // Adjust viewport size as needed

    const currentDirectory = __dirname;
    const htmlFilePath = path.join(currentDirectory, `${fileName}.html`);
    await page.goto(`file://${htmlFilePath}`, { waitUntil: 'networkidle2' });

    const pngFilePath = path.join(currentDirectory, `${fileName}.png`);
    await page.screenshot({ path: pngFilePath, fullPage: true });

    await browser.close();

    console.log(`Screenshot saved to ${pngFilePath}`);
  } catch (error) {
    console.error('Error taking screenshot:', error);
  }
}

// Check if the script is being run directly
if (require.main === module) {
  const fileName = process.argv[2]; // Get the file name from the command line
  if (fileName) {
    html_to_png(fileName);
  } else {
    console.error('Please provide a filename as an argument.');
  }
}

// Export the function for use in other modules
module.exports = html_to_png;


In [ ]:
%%writefile html_to_png.py
import subprocess

def html_to_png(html_file_name):
    """
    Converts an HTML file to a PNG file using a Node.js script.

    Args:
        html_file_name (str): The name of the HTML file (without the .html extension).
    """
    try:
        # Run the Node.js script with the HTML file name as an argument
        result = subprocess.run(
            ['node', 'screenshot.js', html_file_name],
            check=True,  # Raise an error if the command fails
            text=True,   # Capture output as text
            capture_output=True
        )
        print(f"Successfully converted {html_file_name}.html to {html_file_name}.png")
        print(result.stdout)  # Print the output of the Node.js script
    except subprocess.CalledProcessError as e:
        print(f"Error converting {html_file_name}.html to PNG:")
        print(e.stderr)  # Print the error message from the Node.js script

# Example usage
if __name__ == "__main__":
    html_to_png('manhattan')
